### 同步调用

In [9]:
from ollama import Client
client = Client(
)
response = client.chat(model='qwen2.5:0.5b', messages=[
  {
    'role': 'user',
    'content': '介绍一下你自己，少于20字，要求回复3种答案，使用json格式返回',
  },
])

print(response.get("message")["content"])

```json
{
  "answer1": "我是阿里云自主研发的大规模语言模型Qwen，能够回答问题、创作文字和提供文本摘要。",
  "answer2": "我是一种预训练的语言模型，通过大规模的标注数据学习并理解自然语言。",
  "answer3": "我叫通义千问，是由阿里云团队开发的AI助手。"
}
```


### 异步调用

In [2]:
import asyncio
from ollama import AsyncClient

async def chat():
    # 创建异步客户端实例
    client = AsyncClient(
        host='http://localhost:11434',
        headers={'x-some-header': 'some-value'}
    )
    
    # 发送异步请求并获取响应
    response = await client.chat(model='llama3.2-vision', messages=[
        {
            'role': 'user',
            'content': 'Why is the sky blue?',
        },
    ])
    
    # 输出 response 中的 content
    print(response.get("message")["content"])


# 在 Jupyter Notebook 中直接运行
await chat()

# # 运行异步函数
# if __name__ == "__main__":
#     asyncio.run(main())

The sky appears blue to us because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. Here's what happens:

1. **Sunlight enters Earth's atmosphere**: When sunlight enters our atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2).
2. **Scattering occurs**: These gas molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths.
3. **Blue light is scattered in all directions**: As a result, the blue light is distributed throughout the atmosphere, giving the sky its blue appearance.

Here are some interesting facts to help you understand this phenomenon better:

* The scattering effect is more pronounced for shorter wavelengths like blue and violet, which is why the sky typically appears blue during the daytime.
* At sunrise and sunset, the sky can take on hues of red, orange, and pink because the light has to travel through more of the atmosphere to

### 流式响应

In [3]:
import asyncio
from ollama import AsyncClient

async def chat():
    client = AsyncClient(
        host='http://localhost:11434',
        headers={'x-some-header': 'some-value'}
    )
    message = {'role': 'user', 'content': 'Why is the sky blue?'}
    async for part in await client.chat(model='llama3.2-vision', messages=[message], stream=True):
        print(part['message']['content'], end='', flush=True)

# if __name__ == "__main__":
#     asyncio.run(chat())

await chat()

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first explained it in the late 19th century.

Here's what happens:

1. **Sunlight enters Earth's atmosphere**: When sunlight enters our atmosphere, it consists of all colors of the visible spectrum, including red, orange, yellow, green, blue, indigo, and violet.
2. **Molecules scatter light**: The tiny molecules in the air (like nitrogen and oxygen) scatter the light in all directions. This scattering effect is more pronounced for shorter wavelengths of light, like blue and violet.
3. **Blue light is scattered more**: Because blue light has a shorter wavelength (around 450-495 nanometers), it is scattered even more than other colors by the tiny molecules in the air. This means that blue light is dispersed throughout the atmosphere.
4. **Our eyes perceive the blue color**: When we look up at the sky, our eyes receive the scattered blue light from all directions. 

### 多模态输入
![xxx](/disk1/shuxun/Dataset/NFT1000/0xAzuki/img/0xAzuki_1.png)

In [5]:
import asyncio
from ollama import AsyncClient
import base64

async def chat(prompt: str, image_base64: str = None):
    """
    多模态聊天入口，接收用户的文本提示和可选的图片 base64 编码。
    
    Args:
        prompt (str): 用户的文本输入。
        image_base64 (str, optional): 图片的 base64 编码字符串，默认为 None。
    """
    # 创建异步客户端
    client = AsyncClient(
        host='http://localhost:11434',
        headers={'x-some-header': 'some-value'}
    )
    
    # 构造消息
    message = {
        'role': 'user',
        'content': prompt
    }
    
    # 如果提供了图片 base64 编码，则添加到消息中
    if image_base64:
        message['images'] = [image_base64]
    
    # 发送流式请求并处理响应
    try:
        async for part in await client.chat(model='llama3.2-vision', messages=[message], stream=True):
            content = part.get('message', {}).get('content', '')
            print(content, end='', flush=True)
    except Exception as e:
        print(f"\nError: {e}")

# 示例调用（在 Jupyter Notebook 中）
# prompt = "describe the picture as detail as possible"
prompt = "给我用中文描述一下这幅图片"
with open("/disk1/shuxun/Dataset/NFT1000/0xAzuki/img/0xAzuki_1.png", "rb") as image_file:  # 替换为你的图像路径
    image_base64 = base64.b64encode(image_file.read()).decode("utf-8")

await chat(prompt, image_base64)

这个图像是漫画风格的女性头像，背景是黄色。她的头发是红色的，并且是半捆式，打了一个小小的马尾。她的眼睛是深棕色的，她的脸上没有表情。但是，她的嘴巴却是轻松开着的，看起来她正在吸烟或吃东西。她的耳朵很大，显得有些奇怪。她穿着一件白色短袖衬衫，衬衫敞开，露出一个白色的内衣。她的上身和头部被背景遮住了。她似乎是日本漫画风格的角色的特征，比如她的眼睛、鼻子和嘴巴的形状，以及她戴的耳环。整体来说，这幅图像是一个简洁但有趣的漫画风格人物设计。

### 多轮对话

In [6]:
import asyncio
from ollama import AsyncClient

async def chat_with_history():
    """
    支持多轮对话的聊天入口，维护对话历史并支持多模态输入。
    """
    # 创建异步客户端
    client = AsyncClient(
        host='http://localhost:11434',
        headers={'x-some-header': 'some-value'}
    )
    
    # 初始化对话历史
    history = []
    
    while True:
        # 获取用户输入
        prompt = input("You: ")
        if prompt.lower() in ['exit', 'quit']:  # 输入 exit 或 quit 退出
            print("Goodbye!")
            break
        
        # （可选）获取图片 base64 编码
        image_base64 = None
        use_image = input("Do you want to include an image? (yes/no): ").lower()
        if use_image == 'yes':
            image_path = input("Enter image path: ")
            try:
                with open(image_path, "rb") as image_file:
                    image_base64 = base64.b64encode(image_file.read()).decode("utf-8")
            except FileNotFoundError:
                print("Error: Image file not found, proceeding without image")
        
        # 构造当前消息
        current_message = {
            'role': 'user',
            'content': prompt
        }
        if image_base64:
            current_message['images'] = [image_base64]
        
        # 将当前消息添加到历史
        history.append(current_message)
        
        # 发送流式请求，包含完整历史
        print("Assistant: ", end='', flush=True)
        try:
            async for part in await client.chat(model='llama3.2-vision', messages=history, stream=True):
                content = part.get('message', {}).get('content', '')
                print(content, end='', flush=True)
            
            # 将模型响应添加到历史
            assistant_message = {
                'role': 'assistant',
                'content': content
            }
            history.append(assistant_message)
            print()  # 换行
        except Exception as e:
            print(f"\nError: {e}")
            break

# 在 Jupyter Notebook 中运行
await chat_with_history()

Assistant: 您好！有什么可以帮助您的吗？
Assistant: 北京！中国的政治和文化中心。天安门、故宫、颐和园这些景点一定是你去过的地方了吧？又或者你正在享受美食，品尝著北京的小吃，如饺子、烤鸭等？有何打算呢？
Assistant: 你刚才说了：你现在在北京！
Goodbye!
